In [1]:
%%bash

# Test Run with 10 designs 
target="PDL1.json"
target_name="${target%.json}"  # Strip the .json extension
target_path="/workspace/settings_target/$target"
log_dir="/workspace/outputs/$target_name"
log_file="$log_dir/${target_name}-bindcraft_log.txt"

# Ensure log directory exists
mkdir -p "$log_dir"

echo "[INFO] Checking target settings file..."
if [ ! -f "$target_path" ]; then
  echo "[ERROR] Target file '$target' not found in /workspace/settings_target/"
  exit 1
fi

echo "[INFO] Starting BindCraft with target '$target'"
echo "[INFO] Logging output to $log_file"

nohup python /app/bindcraft/bindcraft.py \
  --settings "$target_path" \
  --filters /workspace/settings_filters/default_filters.json \
  --advanced /workspace/settings_advanced/default_4stage_multimer.json \
  > "$log_file" 2>&1 &

echo "[INFO] BindCraft is now running in the background (PID=$!)"

mkdir: cannot create directory '/workspace': Permission denied


[INFO] Checking target settings file...
[ERROR] Target file 'PDL1.json' not found in /workspace/settings_target/


CalledProcessError: Command 'b'\n# Test Run with 10 designs \ntarget="PDL1.json"\ntarget_name="${target%.json}"  # Strip the .json extension\ntarget_path="/workspace/settings_target/$target"\nlog_dir="/workspace/outputs/$target_name"\nlog_file="$log_dir/${target_name}-bindcraft_log.txt"\n\n# Ensure log directory exists\nmkdir -p "$log_dir"\n\necho "[INFO] Checking target settings file..."\nif [ ! -f "$target_path" ]; then\n  echo "[ERROR] Target file \'$target\' not found in /workspace/settings_target/"\n  exit 1\nfi\n\necho "[INFO] Starting BindCraft with target \'$target\'"\necho "[INFO] Logging output to $log_file"\n\nnohup python /app/bindcraft/bindcraft.py \\\n  --settings "$target_path" \\\n  --filters /workspace/settings_filters/default_filters.json \\\n  --advanced /workspace/settings_advanced/default_4stage_multimer.json \\\n  > "$log_file" 2>&1 &\n\necho "[INFO] BindCraft is now running in the background (PID=$!)"\n'' returned non-zero exit status 1.

In [ ]:
!tail -n 50 -f /workspace/outputs/${log_file}

In [1]:
import sys
import os
DEV_DIR = '/home/yarrow/projects/bindcraft-runpod/functions/'
PROD_DIR = '/app/bindcraft/functions'

path_used = DEV_DIR if os.path.exists(DEV_DIR) else PROD_DIR
print(f"Using function path: {path_used}")
sys.path.append(path_used)

from main_UI import launch_all_ui

launch_all_ui()


Using function path: /home/yarrow/projects/bindcraft-runpod/functions/
Step 1: Upload, edit or create a new target JSON file.
Optional: Upload a JSON file to your settings_target directory.


FileUpload(value=(), accept='.json', description='Upload JSON')

Output()

If needed, edit or create a new target JSON file.
Loading editor for: /home/yarrow/projects/bindcraft-runpod/settings_target/PDL1.json


TypeError: main_launch_target_editor() got an unexpected keyword argument 'json_target_path_widget'